In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from transformers import RobertaTokenizer
import torch.nn.functional as F

# Add project root to path
sys.path.insert(0, '/home/paloma/cerebrum-artis')

from cerebrum_artis.models.ensemble.ensemble_v4 import EnsembleV4
from cerebrum_artis.fuzzy.system import FuzzyEmotionSystem

print("✅ Imports loaded successfully")

## 1. Configuração e Carregamento dos Modelos

In [ ]:
# Device
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

# Checkpoints
v2_checkpoint = "/data/paloma/deep-mind-checkpoints/v2_fuzzy_features/checkpoint_best.pt"
v3_checkpoint = "/data/paloma/deep-mind-checkpoints/v3_adaptive_gating/checkpoint_best.pt"

# Load ensemble
print("\n📦 Loading V4 Ensemble...")
ensemble = EnsembleV4(
    v2_checkpoint_path=v2_checkpoint,
    v3_checkpoint_path=v3_checkpoint,
    v2_weight=0.5,
    device=device
)
ensemble.eval()

print("✅ Ensemble loaded successfully")
print(f"   V2 weight: 50% | V3 weight: 50%")

## 2. Preparação de Inputs

In [ ]:
# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Fuzzy system
fuzzy_system = FuzzyEmotionSystem()

# Emotion labels
EMOTIONS = [
    'amusement', 'awe', 'contentment', 'excitement',
    'anger', 'disgust', 'fear', 'sadness', 'something else'
]

print("✅ Transforms and tokenizer ready")

## 3. Input: Imagem e Caption

In [ ]:
# ========================================
# 🎨 EDITE AQUI: Coloque sua imagem e caption
# ========================================

# Exemplo: Picasso - The Pigeon Pea (1912)
image_path = "/home/paloma/cerebrum-artis/garbage/test_images/pablo-picasso_the-pigeon-pea-1912.jpg"
caption = "Fragmented geometric forms with muted earth tones showing analytical cubist style"

# Load and display image
image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(device)

plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title(f"INPUT: {os.path.basename(image_path)}\nCaption: '{caption}'", fontsize=10)
plt.tight_layout()
plt.show()

print(f"📸 Image: {os.path.basename(image_path)}")
print(f"💬 Caption: {caption}")

## 4. Extração de Fuzzy Features

In [ ]:
# Extract fuzzy features from image
fuzzy_features = fuzzy_system.extract_features_from_image(np.array(image))
fuzzy_tensor = torch.tensor(fuzzy_features, dtype=torch.float32).unsqueeze(0).to(device)

# Display fuzzy features
feature_names = ['texture_roughness', 'symmetry', 'complexity', 'color_harmony', 
                 'saturation', 'color_temperature', 'brightness']

fig, ax = plt.subplots(figsize=(10, 4))
colors = ['cyan' if f < 0.5 else 'salmon' for f in fuzzy_features]
bars = ax.barh(feature_names, fuzzy_features, color=colors, edgecolor='black')
ax.set_xlim(0, 1)
ax.set_xlabel('Feature Value', fontsize=12)
ax.set_title('FUZZY FEATURES EXTRACTED', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, fuzzy_features)):
    ax.text(val + 0.02, i, f'{val:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📊 Fuzzy Features:")
for name, val in zip(feature_names, fuzzy_features):
    print(f"   {name}: {val:.3f}")

## 5. Tokenização do Texto

In [ ]:
# Tokenize caption
encoded = tokenizer(
    caption,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

input_ids = encoded['input_ids'].to(device)
attention_mask = encoded['attention_mask'].to(device)

print(f"✅ Text tokenized: {input_ids.shape}")

## 6. Inferência: V2, V3 e V4 Ensemble

In [ ]:
# Run ensemble inference
with torch.no_grad():
    ensemble_logits, v2_logits, v3_logits = ensemble(
        image_tensor, input_ids, attention_mask, fuzzy_tensor
    )
    
    # Convert to probabilities
    v2_probs = F.softmax(v2_logits, dim=1).cpu().numpy()[0]
    v3_probs = F.softmax(v3_logits, dim=1).cpu().numpy()[0]
    ensemble_probs = F.softmax(ensemble_logits, dim=1).cpu().numpy()[0]
    
    # Get predictions
    v2_pred = EMOTIONS[v2_probs.argmax()]
    v3_pred = EMOTIONS[v3_probs.argmax()]
    ensemble_pred = EMOTIONS[ensemble_probs.argmax()]
    
    # Get top-3 confidences
    v2_top3_idx = v2_probs.argsort()[-3:][::-1]
    v3_top3_idx = v3_probs.argsort()[-3:][::-1]
    ensemble_top3_idx = ensemble_probs.argsort()[-3:][::-1]

print("\n🔮 PREDICTIONS:")
print(f"   V2 (Fuzzy Features):  {v2_pred} ({v2_probs.max()*100:.1f}%)")
print(f"   V3 (Adaptive Gating): {v3_pred} ({v3_probs.max()*100:.1f}%)")
print(f"   V4 (Ensemble):        {ensemble_pred} ({ensemble_probs.max()*100:.1f}%)")

print("\n🏆 V2 Top-3 Emotions:")
for idx in v2_top3_idx:
    print(f"   • {EMOTIONS[idx]}: {v2_probs[idx]*100:.1f}%")

print("\n🏆 V3 Top-3 Emotions:")
for idx in v3_top3_idx:
    print(f"   • {EMOTIONS[idx]}: {v3_probs[idx]*100:.1f}%")

print("\n🏆 V4 Ensemble Top-3 Emotions:")
for idx in ensemble_top3_idx:
    print(f"   • {EMOTIONS[idx]}: {ensemble_probs[idx]*100:.1f}%")

## 7. Análise de Agreement (Cosine Similarity)

In [ ]:
# Calculate cosine similarity between V2 and V3 predictions
v2_norm = v2_probs / (np.linalg.norm(v2_probs) + 1e-8)
v3_norm = v3_probs / (np.linalg.norm(v3_probs) + 1e-8)
agreement = np.dot(v2_norm, v3_norm)

print(f"\n🔍 Agreement Analysis: Cosine Similarity = {agreement:.3f}")
print(f"   Interpretation:")
if agreement > 0.8:
    print(f"   ✅ HIGH agreement - Models strongly agree")
elif agreement > 0.6:
    print(f"   ✓ MODERATE agreement - Models somewhat agree")
else:
    print(f"   ⚠️  LOW agreement - Models disagree")

# Calculate adaptive gating weight (como no V3)
max_alpha = 0.8
min_alpha = 0.2
alpha = max_alpha - (max_alpha - min_alpha) * agreement

print(f"\n⚖️  Adaptive Gating Metrics (V3 internal):")
print(f"   Agreement (cosine sim): {agreement:.3f}")
print(f"   Adaptive Alpha (α): {alpha:.3f}")
print(f"   Weights:")
print(f"   • Neural: {alpha*100:.1f}%")
print(f"   • Fuzzy:  {(1-alpha)*100:.1f}%")

## 8. Visualização: Comparação de Predições

In [ ]:
# Create comparison plot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# V2 predictions
axes[0].barh(EMOTIONS, v2_probs, color='steelblue', edgecolor='black')
axes[0].set_xlim(0, 1)
axes[0].set_xlabel('Probability', fontsize=12)
axes[0].set_title(f'Neural Network\n→ {v2_pred} ({v2_probs.max()*100:.1f}%)', 
                  fontsize=14, fontweight='bold', color='steelblue')
axes[0].grid(axis='x', alpha=0.3)

# V3 predictions
axes[1].barh(EMOTIONS, v3_probs, color='darkorange', edgecolor='black')
axes[1].set_xlim(0, 1)
axes[1].set_xlabel('Probability', fontsize=12)
axes[1].set_title(f'iFeature Values\n→ {v3_pred} ({v3_probs.max()*100:.1f}%)', 
                  fontsize=14, fontweight='bold', color='darkorange')
axes[1].set_yticklabels([])
axes[1].grid(axis='x', alpha=0.3)

# V4 Ensemble predictions
axes[2].barh(EMOTIONS, ensemble_probs, color='mediumseagreen', edgecolor='black')
axes[2].set_xlim(0, 1)
axes[2].set_xlabel('Probability', fontsize=12)
axes[2].set_title(f'V3 Final (α={alpha:.2f})\n→ {ensemble_pred} ({ensemble_probs.max()*100:.1f}%)', 
                  fontsize=14, fontweight='bold', color='mediumseagreen')
axes[2].set_yticklabels([])
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Visualização Completa (Similar ao Exemplo)

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.3)

# ===== TOP: Image and Fuzzy Features =====
ax_img = fig.add_subplot(gs[0, 0:2])
ax_img.imshow(image)
ax_img.axis('off')
ax_img.set_title(f"INPUT: {os.path.basename(image_path)}\nCaption: '{caption}'", 
                 fontsize=12, fontweight='bold')

ax_fuzzy = fig.add_subplot(gs[0, 2])
colors_fuzzy = ['cyan' if f < 0.5 else 'salmon' for f in fuzzy_features]
ax_fuzzy.barh(feature_names, fuzzy_features, color=colors_fuzzy, edgecolor='black')
ax_fuzzy.set_xlim(0, 1)
ax_fuzzy.set_xlabel('Value', fontsize=10)
ax_fuzzy.set_title('FUZZY FEATURES EXTRACTED', fontsize=12, fontweight='bold')
ax_fuzzy.grid(axis='x', alpha=0.3)
for i, val in enumerate(fuzzy_features):
    ax_fuzzy.text(val + 0.02, i, f'{val:.3f}', va='center', fontsize=9)

# ===== MIDDLE: Predictions V2, V3, V4 =====
ax_v2 = fig.add_subplot(gs[1, 0])
ax_v2.barh(EMOTIONS, v2_probs, color='steelblue', edgecolor='black')
ax_v2.set_xlim(0, 1)
ax_v2.set_xlabel('Probability', fontsize=10)
ax_v2.set_title(f'Neural Network\n→ {v2_pred} ({v2_probs.max()*100:.1f}%)', 
                fontsize=12, fontweight='bold', color='steelblue')
ax_v2.grid(axis='x', alpha=0.3)

ax_v3 = fig.add_subplot(gs[1, 1])
ax_v3.barh(EMOTIONS, v3_probs, color='darkorange', edgecolor='black')
ax_v3.set_xlim(0, 1)
ax_v3.set_xlabel('Probability', fontsize=10)
ax_v3.set_title(f'iFeature Values\n→ {v3_pred} ({v3_probs.max()*100:.1f}%)', 
                fontsize=12, fontweight='bold', color='darkorange')
ax_v3.set_yticklabels([])
ax_v3.grid(axis='x', alpha=0.3)

ax_v4 = fig.add_subplot(gs[1, 2])
ax_v4.barh(EMOTIONS, ensemble_probs, color='mediumseagreen', edgecolor='black')
ax_v4.set_xlim(0, 1)
ax_v4.set_xlabel('Probability', fontsize=10)
ax_v4.set_title(f'V3 Final (α={alpha:.2f})\n→ {ensemble_pred} ({ensemble_probs.max()*100:.1f}%)', 
                fontsize=12, fontweight='bold', color='mediumseagreen')
ax_v4.set_yticklabels([])
ax_v4.grid(axis='x', alpha=0.3)

# ===== BOTTOM: Agreement Analysis =====
ax_agreement = fig.add_subplot(gs[2, :])

x = np.arange(len(EMOTIONS))
width = 0.25

ax_agreement.bar(x - width, v2_probs, width, label='Neural', color='steelblue', edgecolor='black')
ax_agreement.bar(x, v3_probs, width, label='Fuzzy', color='darkorange', edgecolor='black')
ax_agreement.bar(x + width, ensemble_probs, width, label='V3 Final', color='mediumseagreen', edgecolor='black')

ax_agreement.set_ylabel('Probability', fontsize=12)
ax_agreement.set_xlabel('Emotions', fontsize=12)
ax_agreement.set_title(f'Agreement Analysis: Cosine Similarity = {agreement:.3f}', 
                       fontsize=14, fontweight='bold')
ax_agreement.set_xticks(x)
ax_agreement.set_xticklabels(EMOTIONS, rotation=45, ha='right')
ax_agreement.legend(loc='upper right', fontsize=11)
ax_agreement.grid(axis='y', alpha=0.3)
ax_agreement.set_ylim(0, max(v2_probs.max(), v3_probs.max(), ensemble_probs.max()) * 1.1)

# Add text box with metrics
metrics_text = f"""ADAPTIVE GATING METRICS

Agreement (cosine sim):
{agreement:.3f}

Adaptive Weight Alpha (α):
{alpha:.3f}

Weights:
• Neural: {alpha*100:.1f}%
• Fuzzy: {(1-alpha)*100:.1f}%

Top 3 Emotions (Final):
• {EMOTIONS[ensemble_top3_idx[0]]}: {ensemble_probs[ensemble_top3_idx[0]]*100:.1f}%
• {EMOTIONS[ensemble_top3_idx[1]]}: {ensemble_probs[ensemble_top3_idx[1]]*100:.1f}%
• {EMOTIONS[ensemble_top3_idx[2]]}: {ensemble_probs[ensemble_top3_idx[2]]*100:.1f}%
"""

ax_agreement.text(1.02, 0.5, metrics_text, transform=ax_agreement.transAxes,
                  fontsize=10, verticalalignment='center',
                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('V4 ENSEMBLE PREDICTION ANALYSIS', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

## 10. Resumo Final

In [ ]:
print("="*80)
print("📊 V4 ENSEMBLE - PREDICTION SUMMARY")
print("="*80)
print(f"\n🎨 INPUT:")
print(f"   Image: {os.path.basename(image_path)}")
print(f"   Caption: '{caption}'")

print(f"\n🔮 PREDICTIONS:")
print(f"   V2 (Concatenation):   {v2_pred:20s} confidence={v2_probs.max()*100:5.1f}%")
print(f"   V3 (Adaptive Gating): {v3_pred:20s} confidence={v3_probs.max()*100:5.1f}%")
print(f"   V4 (Ensemble):        {ensemble_pred:20s} confidence={ensemble_probs.max()*100:5.1f}%")

print(f"\n🔍 AGREEMENT:")
print(f"   Cosine Similarity: {agreement:.3f}")
print(f"   Status: {'✅ HIGH' if agreement > 0.8 else '✓ MODERATE' if agreement > 0.6 else '⚠️ LOW'}")

print(f"\n⚖️  ENSEMBLE WEIGHTS:")
print(f"   V2: 50.0%")
print(f"   V3: 50.0%")

print(f"\n🏆 FINAL PREDICTION: {ensemble_pred.upper()}")
print(f"   Confidence: {ensemble_probs.max()*100:.1f}%")
print("\n" + "="*80)